# Working with DataFrames

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_columns', 50)
%matplotlib inline
import os
os.getcwd()

'C:\\Users\\chris.jabr\\PycharmProjects\\Python-Library\\Pandas'

### References

[Official Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/api.html#dataframe)

[Greg Reda Intro](http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/)

[R Comparison](https://pandas.pydata.org/pandas-docs/stable/comparison_with_r.html)

[Modern Pandas](https://github.com/TomAugspurger/effective-pandas/blob/master/modern_1_intro.ipynb)

[Pandas Cheat Sheet](http://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

## Import / Create Data

In [3]:
# import from jupyterLab browser
from_csv = pd.read_csv('../Data/mariano-rivera.csv')
football = pd.read_excel('../football_data.xlsx')

# import from pycharm (dir changes randomly...)
# from_csv = pd.read_csv(r"C:\Users\Kelley\PycharmProjects\Python-Library\Data\mariano-rivera.csv")
# football = pd.read_excel('football_data.xlsx')


# Create DF w/ Dict
extradata = pd.DataFrame({'year': [2013, 2014, 2013],
        'team': ['Wily Coyotes', None, 'Wily Coyotes'],  # None = NULL
        'wins': [15, 16, 15],
        'losses': [None, 3, None]})  # NaN = NULL for numeric columns

# Create DF w/ list of tuples
moredata = pd.DataFrame.from_records([('Bears', 'brown'), ('Packers', 'green'), ('Lions', 'blue')], 
                                    columns = ['team', 'color'])

# Create DF from Series
notes = pd.Series(["this one wasn't good", "NCR-12345 has a problem", "zip code was at 10101 and ticket #12345", "Bob and bobbert had a good time", "mascot was there", 
                   "had a free drink", "", "Pandas are fun. Mr Bob would know.", "i think this is a fake team", "missing team"])
notes_df = pd.DataFrame(notes)
football

,year,team,wins,losses
0,2010,Bears,11,5
1,2011,Bears,8,8
2,2012,Bears,10,6
3,2011,Packers,15,1
4,2012,Packers,11,5
5,2010,Lions,6,10
6,2011,Lions,10,6
7,2012,Lions,4,12


# FROM

### Concat (rowbind/UNION ALL, columnbind)

In [4]:
append_rows = pd.concat([football, extradata], ignore_index = True)  # (SQL UNION ALL) new table index
append_cols = pd.concat([append_rows, notes_df], axis = 1)  # default joins on index
# same as
join_on_index = append_rows.join(notes_df)

football = join_on_index.copy(); football

,year,team,wins,losses,0
0,2010,Bears,11,5.0,this one wasn't good
1,2011,Bears,8,8.0,NCR-12345 has a problem
2,2012,Bears,10,6.0,zip code was at 10101 and ticket #12345
3,2011,Packers,15,1.0,Bob and bobbert had a good time
4,2012,Packers,11,5.0,mascot was there
5,2010,Lions,6,10.0,had a free drink
6,2011,Lions,10,6.0,
7,2012,Lions,4,12.0,Pandas are fun. Mr Bob would know.
8,2013,Wily Coyotes,15,NaN,i think this is a fake team
9,2014,None,16,3.0,missing team


### Merge (SQL Join)

In [5]:
inner_join = pd.merge(football, moredata)
inner_join_2 = football.merge(moredata)
left_join = pd.merge(football, moredata, on='team', how='left')
left_join_2 = football.merge(moredata, how='left')

f = left_join.copy()  # type: pandas.DataFrame
f

,year,team,wins,losses,0,color
0,2010,Bears,11,5.0,this one wasn't good,brown
1,2011,Bears,8,8.0,NCR-12345 has a problem,brown
2,2012,Bears,10,6.0,zip code was at 10101 and ticket #12345,brown
3,2011,Packers,15,1.0,Bob and bobbert had a good time,green
4,2012,Packers,11,5.0,mascot was there,green
5,2010,Lions,6,10.0,had a free drink,blue
6,2011,Lions,10,6.0,,blue
7,2012,Lions,4,12.0,Pandas are fun. Mr Bob would know.,blue
8,2013,Wily Coyotes,15,NaN,i think this is a fake team,NaN
9,2014,None,16,3.0,missing team,NaN


### Explore

In [10]:
f.head()
f.shape  # row/col
f.sample(2)  # random sample rows
f.dtypes
f.isnull()
f.info
f.describe()  # statistics summary

f['team'].value_counts()
f.columns  # columns object
list(f)    # columns list

Index(['year', 'team', 'wins', 'losses', 0, 'color'], dtype='object')

# SELECT

In [30]:
# select
f[['year', 'team']]         # same
f.filter(['year', 'team'])  # same
f.filter(regex='\d')        # select matching pattern

# select & rename
f.rename(columns={0: "my_notes", 'color':'TEAM_COLORS'})
f.filter(['year', 'team']).rename(columns={'team': 'MY_TEAM'})

,year,MY_TEAM
0,2010,Bears
1,2011,Bears
2,2012,Bears
3,2011,Packers
4,2012,Packers
5,2010,Lions
6,2011,Lions
7,2012,Lions
8,2013,Wily Coyotes
9,2014,None


# WHERE

### Filter by Values

In [64]:
f[(f['year'] > 2011) & (f['team'] != 'Bears')]

,year,team,wins,losses,0
4,2012,Packers,11,5.0,mascot was there
7,2012,Lions,4,12.0,Pandas are fun. Mr Bob would know.
8,2013,Wily Coyotes,15,NaN,i think this is a fake team
9,2014,None,16,3.0,missing team
10,2013,Wily Coyotes,15,NaN,NaN


### Filter out duplicate rows

In [72]:
f.drop_duplicates(['year', 'team'])  # based on [optional] columns

,year,team,wins,losses,0
0,2010,Bears,11,5.0,this one wasn't good
1,2011,Bears,8,8.0,NCR-12345 has a problem
2,2012,Bears,10,6.0,zip code was at 10101 and ticket #12345
3,2011,Packers,15,1.0,Bob and bobbert had a good time
4,2012,Packers,11,5.0,mascot was there
5,2010,Lions,6,10.0,had a free drink
6,2011,Lions,10,6.0,
7,2012,Lions,4,12.0,Pandas are fun. Mr Bob would know.
8,2013,Wily Coyotes,15,NaN,i think this is a fake team
9,2014,None,16,3.0,missing team


### Filter in/out NULLS (None, NaN)

In [81]:
f[f['team'].notnull()]

,year,team,wins,losses,0
0,2010,Bears,11,5.0,this one wasn't good
1,2011,Bears,8,8.0,NCR-12345 has a problem
2,2012,Bears,10,6.0,zip code was at 10101 and ticket #12345
3,2011,Packers,15,1.0,Bob and bobbert had a good time
4,2012,Packers,11,5.0,mascot was there
5,2010,Lions,6,10.0,had a free drink
6,2011,Lions,10,6.0,
7,2012,Lions,4,12.0,Pandas are fun. Mr Bob would know.
8,2013,Wily Coyotes,15,NaN,i think this is a fake team
10,2013,Wily Coyotes,15,NaN,NaN


In [ ]:
f[f['losses'].isnull()]

,year,team,wins,losses,0
8,2013,Wily Coyotes,15,NaN,i think this is a fake team
10,2013,Wily Coyotes,15,NaN,NaN


# GROUP BY & Aggregates

In [8]:
f.groupby('color')['wins'].agg(np.sum)  # group by color, sum wins
f.groupby('team')['wins', 'losses'].agg(np.mean)  # average wins & losses by team

,wins,losses
team,,
Bears,9.666667,6.333333
Lions,6.666667,9.333333
Packers,13.000000,3.000000
Wily Coyotes,15.000000,NaN


In [11]:
# more data for grouping
more_categories = pd.DataFrame({'team': ['Bears', 'Lions', 'Packers', 'Wily Coyotes'], 'fan_of': [1,1,0,1]})
more_categories

df = pd.merge(f, more_categories)
df

df.groupby(['fan_of', 'team'])['wins', 'losses'].agg([np.sum, np.mean])
df.groupby(['fan_of', 'team'])['wins', 'losses'].agg({'wins': [np.sum, np.mean], 'losses': np.mean})  # group by 2 cols. agg 2 other cols. map aggregations to cols.

wins               losses
                     sum       mean      mean
fan_of team                                  
0      Packers        26  13.000000  3.000000
1      Bears          29   9.666667  6.333333
       Lions          20   6.666667  9.333333
       Wily Coyotes   30  15.000000       NaN

# ORDER BY

In [12]:
df2 = df.copy()
df2.sort_values(['year', 'team'], ascending=[False, False])  # multiple columns, & desc for each
df2.sort_values('losses', na_position='first')  # nas at top

,year,team,wins,losses,0,color,fan_of
8,2013,Wily Coyotes,15,NaN,i think this is a fake team,NaN,1
9,2013,Wily Coyotes,15,NaN,NaN,NaN,1
3,2011,Packers,15,1.0,Bob and bobbert had a good time,green,0
0,2010,Bears,11,5.0,this one wasn't good,brown,1
4,2012,Packers,11,5.0,mascot was there,green,0
2,2012,Bears,10,6.0,zip code was at 10101 and ticket #12345,brown,1
6,2011,Lions,10,6.0,,blue,1
1,2011,Bears,8,8.0,NCR-12345 has a problem,brown,1
5,2010,Lions,6,10.0,had a free drink,blue,1
7,2012,Lions,4,12.0,Pandas are fun. Mr Bob would know.,blue,1


# DELETE

In [13]:
df2 = df.copy()
df2.drop(['wins'], axis=1)  # same 
df2.drop(columns=['wins'])  # same

,year,team,losses,0,color,fan_of
0,2010,Bears,5.0,this one wasn't good,brown,1
1,2011,Bears,8.0,NCR-12345 has a problem,brown,1
2,2012,Bears,6.0,zip code was at 10101 and ticket #12345,brown,1
3,2011,Packers,1.0,Bob and bobbert had a good time,green,0
4,2012,Packers,5.0,mascot was there,green,0
5,2010,Lions,10.0,had a free drink,blue,1
6,2011,Lions,6.0,,blue,1
7,2012,Lions,12.0,Pandas are fun. Mr Bob would know.,blue,1
8,2013,Wily Coyotes,NaN,i think this is a fake team,NaN,1
9,2013,Wily Coyotes,NaN,NaN,NaN,1


### UPDATE

In [14]:
##  !! change column values
df2 = df.copy()
df2.loc[df2['wins'] < 10, 'wins'] = 5  # somehow updating original df object?
df2

,year,team,wins,losses,0,color,fan_of
0,2010,Bears,11,5.0,this one wasn't good,brown,1
1,2011,Bears,5,8.0,NCR-12345 has a problem,brown,1
2,2012,Bears,10,6.0,zip code was at 10101 and ticket #12345,brown,1
3,2011,Packers,15,1.0,Bob and bobbert had a good time,green,0
4,2012,Packers,11,5.0,mascot was there,green,0
5,2010,Lions,5,10.0,had a free drink,blue,1
6,2011,Lions,10,6.0,,blue,1
7,2012,Lions,5,12.0,Pandas are fun. Mr Bob would know.,blue,1
8,2013,Wily Coyotes,15,NaN,i think this is a fake team,NaN,1
9,2013,Wily Coyotes,15,NaN,NaN,NaN,1


In [15]:
df2 = df.copy()
df2

,year,team,wins,losses,0,color,fan_of
0,2010,Bears,11,5.0,this one wasn't good,brown,1
1,2011,Bears,8,8.0,NCR-12345 has a problem,brown,1
2,2012,Bears,10,6.0,zip code was at 10101 and ticket #12345,brown,1
3,2011,Packers,15,1.0,Bob and bobbert had a good time,green,0
4,2012,Packers,11,5.0,mascot was there,green,0
5,2010,Lions,6,10.0,had a free drink,blue,1
6,2011,Lions,10,6.0,,blue,1
7,2012,Lions,4,12.0,Pandas are fun. Mr Bob would know.,blue,1
8,2013,Wily Coyotes,15,NaN,i think this is a fake team,NaN,1
9,2013,Wily Coyotes,15,NaN,NaN,NaN,1
